<a href="https://colab.research.google.com/github/stef1927/dl-samples/blob/master/linear-regression/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression

This is a notebook for reviewing the basics of linear regression and how it can be implemented from first principles in numpy and pytorch.

Sources:

* https://www.kaggle.com/aakashns/pytorch-basics-linear-regression-from-scratch
* https://nbviewer.org/url/www.cs.toronto.edu/~frossard/post/linear_regression/Linear%20Regression.ipynb

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Create the data

In [2]:
# Input (temp, rainfall, humidity)
inputs_orig = np.array([[73, 67, 43], 
                        [91, 88, 64], 
                        [87, 134, 58], 
                        [102, 43, 37], 
                        [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets_orig = np.array([[56, 70], 
                         [81, 101], 
                         [119, 133], 
                         [22, 37], 
                         [103, 119]], dtype='float32')

The model will fit crop yields for apples and oranges by looking at temperature, rainfall and humidity.

| Region  | Temp (F) | Rainfall (mm) | Humidity (%) | Apples (ton) | Oranges (ton) |
| ---   | --- | ----| ---| --- | --- |
|Kanto  | 73  | 67  | 43 | 56  | 70  |
|Johto  | 91  | 88  | 64 | 81  | 101 |
|Hoenn  | 87  | 134 | 58 | 119 | 133 |
|Sinnoh | 102 | 43  | 37 | 22  | 37  |
|Unova  | 69  | 96  | 70 | 103 | 119 | 

Each target will be estimated as a weighted sum of the input variables, offset by the bias, where the bias and the weights will be learnt through gradient descent.

$y_{apples} = w_{11}*temp + w_{21} * rainfall + w_{13} * humidity + b_1$

$y_{oranges} = w_{21}*temp + w_{22} * rainfall + w_{23} * humidity + b_2$


# Pytorch from scratch

In [3]:
lr = 1e-5
epochs = 1000
inputs = torch.tensor(inputs_orig, requires_grad=True)
targets = torch.tensor(targets_orig, requires_grad=True)

w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

def model(x):
  return x @ w.t() + b

def mse(t1, t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()

for i in range(epochs):
  #print('i: {} W: {}/{}, b: {}/{}'.format(i, w, w.grad, b, b.grad))
  
  # compute predictions and loss
  preds = model(inputs)
  loss = mse(preds, targets)

  # compute the gradients
  loss.backward()

  # adjust the weights and reset the gradients
  with torch.no_grad():
    w -= lr * w.grad
    b -= lr * b.grad
    w.grad.zero_()
    b.grad.zero_()
  
  if i > 0 and i % 10 == 0:
    print('***Epoch: {}, Loss:{}***'.format(i, loss))
  
print('***Final***\nTargets: {}\nPredictions:{}\nLoss:{}\n'.format(targets, preds, loss))


***Epoch: 10, Loss:3237.90576171875***
***Epoch: 20, Loss:2319.900146484375***
***Epoch: 30, Loss:2046.9105224609375***
***Epoch: 40, Loss:1816.579833984375***
***Epoch: 50, Loss:1613.6754150390625***
***Epoch: 60, Loss:1434.7327880859375***
***Epoch: 70, Loss:1276.893798828125***
***Epoch: 80, Loss:1137.6444091796875***
***Epoch: 90, Loss:1014.7706298828125***
***Epoch: 100, Loss:906.3230590820312***
***Epoch: 110, Loss:810.5841064453125***
***Epoch: 120, Loss:726.0416259765625***
***Epoch: 130, Loss:651.3638916015625***
***Epoch: 140, Loss:585.3777465820312***
***Epoch: 150, Loss:527.0499267578125***
***Epoch: 160, Loss:475.47052001953125***
***Epoch: 170, Loss:429.8382263183594***
***Epoch: 180, Loss:389.447265625***
***Epoch: 190, Loss:353.6757507324219***
***Epoch: 200, Loss:321.97607421875***
***Epoch: 210, Loss:293.86627197265625***
***Epoch: 220, Loss:268.92132568359375***
***Epoch: 230, Loss:246.76693725585938***
***Epoch: 240, Loss:227.0738067626953***
***Epoch: 250, Loss:209

# Pytorch built-ins

In [4]:
from torch.utils.data import TensorDataset, DataLoader

lr = 1e-5
epochs = 1000
batch_size = 5
inputs = torch.tensor(inputs_orig, requires_grad=True)
targets = torch.tensor(targets_orig, requires_grad=True)

train_ds = TensorDataset(inputs, targets)
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

optimizer = torch.optim.SGD(model.parameters(), lr=lr)
loss_fcn = F.mse_loss

for i in range(epochs):
  for xb, yb in train_dl:
    # compute predictions and loss
    preds = model(xb)
    loss = loss_fcn(preds, yb)

    # compute the gradients
    loss.backward()

    # adjust the weights and reset the gradients
    optimizer.step()
    optimizer.zero_grad()
  
  if i > 0 and i % 10 == 0:
    print('***Epoch: {}, Loss:{}***'.format(i, loss))
  
print('***Final***\nTargets: {}\nPredictions:{}\nLoss:{}\n'.format(targets, preds, loss))

Parameter containing:
tensor([[ 0.3124,  0.5762,  0.0886],
        [ 0.1606, -0.0734, -0.3997]], requires_grad=True)
Parameter containing:
tensor([ 0.2629, -0.1043], requires_grad=True)
***Epoch: 10, Loss:630.4852905273438***
***Epoch: 20, Loss:458.3590393066406***
***Epoch: 30, Loss:407.7231750488281***
***Epoch: 40, Loss:364.97100830078125***
***Epoch: 50, Loss:327.24951171875***
***Epoch: 60, Loss:293.92340087890625***
***Epoch: 70, Loss:264.4695739746094***
***Epoch: 80, Loss:238.4278106689453***
***Epoch: 90, Loss:215.39279174804688***
***Epoch: 100, Loss:195.007568359375***
***Epoch: 110, Loss:176.957763671875***
***Epoch: 120, Loss:160.9663848876953***
***Epoch: 130, Loss:146.78964233398438***
***Epoch: 140, Loss:134.21275329589844***
***Epoch: 150, Loss:123.04647064208984***
***Epoch: 160, Loss:113.12413024902344***
***Epoch: 170, Loss:104.29890441894531***
***Epoch: 180, Loss:96.44148254394531***
***Epoch: 190, Loss:89.43794250488281***
***Epoch: 200, Loss:83.18792724609375***